In [ ]:
import pandas as pd
import re
import nltk

from fuzzywuzzy import process
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

### Realizaremos una revisión y estandarización de categorías para que las mismas se repitan lo menos posible.

In [2]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'metadata_FL_Google.json'
# Lee el archivo JSON de google y carga los datos en un DataFrame
df_metadada_google = pd.read_json(ruta_archivo, encoding='utf-8')


# Especifica la ruta del archivo JSON
ruta_archivo = 'yelp_business_fl.json'
# Lee el archivo JSON de yelp y carga los datos en un DataFrame
df_metadada_yelp = pd.read_json(ruta_archivo, encoding='utf-8')

In [3]:
# Muestra las primeras filas del DataFrame de google para verificar los campos
df_metadada_google.head()

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,MISC,state,estado
0,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[Restaurant],5.0,1,"{'Service options': ['Dine-in', 'Delivery'], '...",None,FL
1,Fresh Point Country Buffet,"Fresh Point Country Buffet, 10525 US-19, Pinel...",0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,[Buffet restaurant],5.0,2,"{'Service options': ['Delivery'], 'Accessibili...",Permanently closed,FL
2,Oneyda's Bakery,"Oneyda's Bakery, 600 Goodlette-Frank Rd #101, ...",0x88dae191ee505917:0x6ba3e25388d3fad4,26.154754,-81.790528,"[Bakery, Deli]",4.6,19,{'Service options': ['Delivery']},Permanently closed,FL
3,Annie's Bake Shoppe,"Annie's Bake Shoppe, 10331 SW 54th St, Miami, ...",0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,25.717416,-80.361945,"[Bakery, Gift basket store]",4.3,3,"{'Service options': ['In-store shopping', 'Tak...",Open ⋅ Closes 4PM,FL
4,Tropical Park Liquors,"Tropical Park Liquors, 7971 SW 40th St Suite #...",0x88d9b99475d9fd7b:0xea6083d207b2471a,25.733914,-80.325216,[Liquor store],4.7,8,"{'Service options': ['In-store shopping'], 'Ac...",Open ⋅ Closes 12AM,FL


In [4]:
# Muestra las primeras filas del DataFrame de yelp para verificar los campos
df_metadada_yelp.head()

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants"
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,10588 109 Street,Edmonton,FL,53.549633,-113.508780,5.0,20,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Coffee & Tea, Cafes, Pets, Restaurants, Pet Ad..."
2,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,FL,39.906295,-86.047463,3.5,257,"{'CoatCheck': 'False', 'Music': '{'dj': False}...","Sports Bars, American (New), American (Traditi..."
3,L_TT0BFmFwORAMaA83K54A,Village Tap Room,838 Broad Ripple Ave,Indianapolis,FL,39.869911,-86.143577,2.5,23,"{'Alcohol': 'u'none'', 'BestNights': '{'monday...","Gastropubs, Cocktail Bars, Beer Bar, Bars, Res..."
4,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,FL,39.954573,-75.194894,3.0,56,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Restaurants, Automotive, Delis, Gas Stations, ..."


### Obtenemos todas las categorías de ambas plataformas

In [5]:
df_exploded = df_metadada_google.explode('category')
# Obtenemos los valores únicos de la columna 'category' en google después de la explosión. 
categorias_google = df_exploded['category'].unique().tolist()


df_metadada_yelp['categories'] = df_metadada_yelp['categories'].str.split(',')
df_exploded = df_metadada_yelp.explode('categories')
# Obtenemos los valores únicos de la columna 'categories' en yelp después de la explosión. 
categorias_yelp = df_exploded['categories'].unique().tolist()


### Cargamos las categorías en una lista, pero evitamos que se dupliquen. Esta columna del dataframe, sera la original para relacionarlas con los restaurantes a futuro.

In [6]:
# Creamos una lista de categorias para unificarlas.
categorias = []


# Recorremos las categorias de google, las tranformamos a minusculas y eliminamos los espacios en blanco al principio de la cadena.
for x in categorias_google:
    # Si la categoria ya se encuentra en la lista, no la agregamos.
    if x.lower().lstrip() not in categorias:
        categorias.append(x.lower().lstrip())


# Recorremos las categorias de yelp, las tranformamos a minusculas y eliminamos los espacios en blanco al principio de la cadena.
for x in categorias_yelp:
    # Si la categoria ya se encuentra en la lista, no la agregamos.
    if x.lower().lstrip() not in categorias:
        categorias.append(x.lower().lstrip())

# Transformamos la lista de categorias en un dataframe.
df_categorias = pd.DataFrame(categorias)

### Las cadenas de texto que coincidentes en un 95% se unifican en una sola.

In [7]:
# Limpiar y estandarizar categorías.
categorias_limpias = []

# Trabajamos con las categorias de la lista creada anteriormente.
for x in categorias:
    #  Cada categoria se tranforma a minúsculas y se quitan caracteres especiales.
    categoria_limpia = re.sub(r'[^a-zA-Z0-9\s]', '', x.lower())
    
    # De la categoria limpia, busca la mejor coincidencia cercana en la lista original.
    resultado = process.extractOne(categoria_limpia, categorias_limpias)
    
    # Verificar si se encontró una coincidencia
    if resultado is not None:
        mejor_coincidencia, umbral = resultado
        # Si la similitud es mayor al umbral, usar la mejor coincidencia.
        if umbral >= 95:
            categorias_limpias.append(mejor_coincidencia)
        else:
            categorias_limpias.append(categoria_limpia)
    else:
        categorias_limpias.append(categoria_limpia)

### Unificamos las categorías que estén en singular y plural

In [8]:
# Inicializar lematizador de WordNet
lemmatizador = WordNetLemmatizer()

# Lematizar y unificar singulares y plurales.
categorias_lematizadas = [lemmatizador.lemmatize(palabra) for palabra in categorias_limpias]

# Agregamos al dataframe, las categorias estandarizadas.
df_categorias['categorias'] = pd.DataFrame(categorias_lematizadas)

### Como resultado final, nos queda un listado de categorías originales y a la par la categoría unificada a la cual pertenece ahora ese restaurante.

In [10]:
df_categorias

,0,categorias
0,restaurant,restaurant
1,buffet restaurant,buffet restaurant
2,bakery,bakery
3,deli,deli
4,gift basket store,gift basket store
...,...,...
1610,damage restoration,damage restoration
1611,hair loss centers,hair loss centers
1612,car window tinting,car window tinting
1613,wigs,wig
